<a href="https://colab.research.google.com/github/papagorgio23/Python101/blob/master/Validation_F%2B_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Validate F+ Lead Scoring 2.0 Model

## Load Data

In [ ]:
# Installing Library
!pip install pydata_google_auth

In [ ]:
# Using GBQ shout Out to Hughes
import pandas_gbq
import pydata_google_auth

SCOPES = [
    'https://www.googleapis.com/auth/cloud-platform',
    'https://www.googleapis.com/auth/drive',
]

credentials = pydata_google_auth.get_user_credentials(
    SCOPES,
    auth_local_webserver=False)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=262006177488-3425ks60hkk80fssi9vpohv88g6q1iqd.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=gEyhUDQUFVmiroOIt451qREg7hQNgE&prompt=consent&access_type=offline
Enter the authorization code: 4/wwF9w2XOQ68My0P2PC1HEs6q7KmkhGatiMs8A9TMzs1FVZ-p8kOIVRk


In [ ]:
sql = """

SELECT 
    a.id
    ,m.name
    ,m.loan_application_id
    ,m.lead_type__c
    ,m.createddate
    ,m.prequal_submitted_date__c
    ,m.full_app_submitted_date__c
    ,m.Loan_Officer_Assigned
    ,m.date_funded__c
    ,m.funded_flag
    ,m.loan_officer_name__c
    ,m.amount_of_loan_requested__c
    ,m.first_amount_of_loan_requested
    ,m.verifiable_annual_income__c
    ,m.first_income_p1
    ,m.co_app_verifiable_annual_income__c
    ,m.first_income_p2
    ,m.co_app
    ,m.first_coapp
    ,m.income_sum
    ,CAST(m.first_income_sum AS INT64) AS first_income_sum
    ,m.c_LTI
    ,m.First_LTI
    ,m.loan_use__c
    ,m.first_loan_use
    ,m.employment_status__c
    ,m.ndi_ratio__c
    ,m.first_ndi_ratio__c
    ,m.fico__c
    ,m.first_FICO
    ,m.utm_source__c
    ,m.bcc0300__c
    ,m.first_bcc0300__c
    ,a.interest_rate__c
    ,a.risk_group__c
    ,a.final_risk_group__c
    ,a.risk_group_p1__c
    
FROM 
    `ffn-dw-bigquery-prd.Credit_Views.Check_Sales_NPV_Model_Inputs` m
    
LEFT JOIN 
    `freedom-dw.salesforce_ffam.application__c` a ON m.name = a.name

WHERE 
    m.createddate BETWEEN '2019-05-01' AND '2019-10-15'
"""

In [ ]:
df1 = pandas_gbq.read_gbq(sql, project_id='ffn-dw-bigquery-prd', credentials=credentials, dialect='standard')

df = df1

In [ ]:
df1.head()

,id,name,loan_application_id,lead_type__c,createddate,prequal_submitted_date__c,full_app_submitted_date__c,Loan_Officer_Assigned,date_funded__c,funded_flag,loan_officer_name__c,amount_of_loan_requested__c,first_amount_of_loan_requested,verifiable_annual_income__c,first_income_p1,co_app_verifiable_annual_income__c,first_income_p2,co_app,first_coapp,income_sum,first_income_sum,c_LTI,First_LTI,loan_use__c,first_loan_use,employment_status__c,ndi_ratio__c,first_ndi_ratio__c,fico__c,first_FICO,utm_source__c,bcc0300__c,first_bcc0300__c,interest_rate__c,risk_group__c,final_risk_group__c,risk_group_p1__c
0,a010f00000TBhXGAA1,App-04935797,3639993.0,Web,2019-06-10 15:00:48+00:00,2019-06-10 15:00:49+00:00,NaT,0,NaT,0,None,20000.0,NaN,60000.0,60000,NaN,None,0,0,60000.0,60000.0,0.333333,0.333333,Debt Consolidation,Debt Consolidation,Retired,NaN,None,NaN,None,credible,NaN,12.0,NaN,NaN,NaN,NaN
1,a010f00000TBhlwAAD,App-04935988,NaN,Phone,2019-06-10 15:15:42+00:00,2019-06-10 15:15:44+00:00,NaT,0,NaT,0,None,22000.0,NaN,78000.0,78000,NaN,None,0,0,78000.0,78000.0,0.282051,0.282051,Debt Consolidation,Debt Consolidation,Full-time,NaN,None,NaN,None,aff_dcg,NaN,NaN,NaN,NaN,NaN,NaN
2,a010f00000Wt7Q1AAJ,App-04846155,3074664.0,Web,2019-06-01 20:04:10+00:00,2019-06-01 20:04:12+00:00,NaT,0,NaT,0,None,10000.0,NaN,108000.0,108000,NaN,None,0,0,108000.0,108000.0,0.092593,0.092593,Debt Consolidation,Debt Consolidation,Full-time,NaN,None,NaN,None,Lending Tree,NaN,7.0,NaN,NaN,NaN,NaN
3,a010f00000ZA8I3AAL,App-06076381,NaN,Web,2019-09-24 19:26:56+00:00,2019-09-24 19:26:59+00:00,NaT,0,NaT,0,None,19000.0,NaN,59000.0,59000,NaN,None,0,0,59000.0,59000.0,0.322034,0.322034,Major Purchase,Major Purchase,Full-time,NaN,None,NaN,None,Personalloanpro,NaN,NaN,NaN,NaN,NaN,NaN
4,a010f00000YlgUmAAJ,App-05905947,NaN,Phone,2019-09-06 19:10:33+00:00,2019-09-06 19:10:35+00:00,NaT,0,NaT,0,None,40000.0,NaN,24000.0,24000,NaN,None,0,0,24000.0,24000.0,1.666667,1.666667,Debt Consolidation,Debt Consolidation,Full-time,NaN,None,NaN,None,Rescue,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.columns

Index(['id', 'name', 'loan_application_id', 'lead_type__c', 'createddate',
       'prequal_submitted_date__c', 'full_app_submitted_date__c',
       'Loan_Officer_Assigned', 'date_funded__c', 'funded_flag',
       'loan_officer_name__c', 'amount_of_loan_requested__c',
       'first_amount_of_loan_requested', 'verifiable_annual_income__c',
       'first_income_p1', 'co_app_verifiable_annual_income__c',
       'first_income_p2', 'co_app', 'first_coapp', 'income_sum',
       'first_income_sum', 'c_LTI', 'First_LTI', 'loan_use__c',
       'first_loan_use', 'employment_status__c', 'ndi_ratio__c',
       'first_ndi_ratio__c', 'fico__c', 'first_FICO', 'utm_source__c',
       'bcc0300__c', 'first_bcc0300__c', 'interest_rate__c', 'risk_group__c',
       'final_risk_group__c', 'risk_group_p1__c'],
      dtype='object')

In [ ]:
df.columns

Index(['name', 'loan_application_id', 'co_app_loan_application_id',
       'lead_type__c', 'createddate', 'prequal_submitted_date__c',
       'full_app_submitted_date__c', 'Loan_Officer_Assigned', 'date_funded__c',
       'funded_flag', 'loan_officer_name__c', 'amount_of_loan_requested__c',
       'first_amount_of_loan_requested', 'verifiable_annual_income__c',
       'first_income_p1', 'co_app_verifiable_annual_income__c',
       'first_income_p2', 'co_app', 'first_coapp', 'income_sum',
       'first_income_sum', 'c_LTI', 'First_LTI', 'loan_use__c',
       'first_loan_use', 'employment_status__c', 'start_employment_status',
       'ndi_ratio__c', 'first_ndi_ratio__c', 'fico__c', 'first_FICO',
       'utm_source__c', 'bcc0300__c', 'first_bcc0300__c'],
      dtype='object')

## Model Test 1 - First touch variables

In [ ]:
# removed duplicate columns and save the first values
firstDf = df.drop(["id", "amount_of_loan_requested__c", "verifiable_annual_income__c", \
                "co_app_verifiable_annual_income__c", "co_app", "c_LTI", \
                "loan_use__c", "employment_status__c", "ndi_ratio__c", \
                "fico__c", "bcc0300__c"], axis=1)

In [ ]:
firstDf.columns

Index(['name', 'loan_application_id', 'lead_type__c', 'createddate',
       'prequal_submitted_date__c', 'full_app_submitted_date__c',
       'Loan_Officer_Assigned', 'date_funded__c', 'funded_flag',
       'loan_officer_name__c', 'first_amount_of_loan_requested',
       'first_income_p1', 'first_income_p2', 'first_coapp', 'income_sum',
       'first_income_sum', 'First_LTI', 'first_loan_use', 'first_ndi_ratio__c',
       'first_FICO', 'utm_source__c', 'first_bcc0300__c', 'interest_rate__c',
       'risk_group__c', 'final_risk_group__c', 'risk_group_p1__c'],
      dtype='object')

## Model Test 2 - Current model

# Mount Drive

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# set working directory
import os
os.chdir("/content/drive/My Drive/Data Scientist/F+ Lead Scoring Model/Validation/")
os.getcwd()

'/content/drive/My Drive/Data Scientist/F+ Lead Scoring Model/Validation'

In [ ]:
ls

 final_df.csv         main_leadLevel.py           scoringData.py
 first_df.csv         model.py                    unitEconomicsData.csv
 Fplus_Lead_Scoring   __pycache__/               'Validation F+ Model.ipynb'
 full_data.csv        Retrain_Fplus.csv
 library_binning.py   score_collector_phase1.py


## Module Setup

In [ ]:
import numpy as np
import pandas as pd
import logging
import os
from sklearn.externals import joblib


#dirdata = os.path.join(os.path.dirname(__file__), 'Data')

global initialized
initialized = False

def init():
    
    global fplus_ls_model
    global unitEconomicsData
    
    global initialized
    if (initialized == True):
        return
    
    try:
        fplus_ls_model = joblib.load('Fplus_Lead_Scoring')
        unitEconomicsData = pd.read_csv('unitEconomicsData.csv')
        #fplus_ls_model = joblib.load(os.path.join(dirdata, 'Fplus_Lead_Scoring'))
        #unitEconomicsData = pd.read_csv(os.path.join(dirdata, 'unitEconomicsData.csv'))
        
    except Exception as e:
        #print(e)
        return
    
    initialized = True



/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
## initialize Model
init()
## doesn't work.... So here we go manual
#fplus_ls_model = joblib.load('Data/Fplus_Lead_Scoring')
#unitEconomicsData = pd.read_csv('Data/unitEconomicsData.csv')

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.logistic module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [ ]:
unitEconomicsData.head()

,utm_source,CPA
0,247,225.00
1,amone,418.59
2,201,351.00
3,222,240.00
4,ck,306.48


## Library Binning

In [ ]:
import numpy as np
import pandas as pd

def get_co_app_cat(co_app_income):
    
    if pd.isnull(co_app_income):
        return 0
    return 1

def get_loan_use_cat(loan_use):
    
    if pd.isnull(loan_use):
        return 3
    
    loan_use = loan_use.strip()
    if (loan_use == 'Credit Card Refinancing'):
        return 4
    if (loan_use in ['Major Purchase','Other']):
        return 2
    if (loan_use == 'Auto Purchase'):
        return 1
    
    return 3

def get_employment_cat(employment_status):
    
    if pd.isnull(employment_status):
        employment_status = ''
    
    employment_status = employment_status.strip()
    if (employment_status == 'Retired'):
        return 4
    if (employment_status in ['Self-employed']):
        return 2
    if (employment_status in ['Other', '']):
        return 1
    
    return 3

def get_loan_amount_cat(loan_amount):
    
    if pd.isnull(loan_amount):
        return 1
    
    loan_amount = float(loan_amount)
    if (loan_amount < 15000):
        return 4
    if (loan_amount >= 15000) and (loan_amount < 20000):
        return 3
    if (loan_amount >= 20000) and (loan_amount < 25000):
        return 2
    
    return 1

def get_mkt_chan_cat(utm_source):
    
    if pd.isnull(utm_source):
        return 3
    
    utm_source = utm_source.strip()
    if (utm_source in ['creditkarma','nerdwallet']):
        return 7
    if (utm_source in ['credible','experian']):
        return 6
    if (utm_source in ['website', 'google','msn','ck','nerd',
                        '115','save','dm','SLH','201']):
        return 5
    if (utm_source in ['facebook', 'even','uplift','Quinstreet',
                       'Personalloanpro','113']):
        return 2
    if (utm_source in ['LendEDU', 'monevo','247','sfl']):
        return 1
    
    return 3

def get_fico(fico):
    
    if pd.isnull(fico):
        return 990
    
    fico = int(fico)
    if (fico >= 9000):
        return 990
    if fico < 600:
        return 990
    
    return fico

def get_lti(lti):
    
    if pd.isnull(lti):
        return 36
    
    lti = float(lti)
    if (lti > 35) or (lti < 1):
        return 36
    if (lti >= 1) and (lti < 2):
        return 35
    if (lti >= 2) and (lti < 3):
        return 34
    
    return np.floor(lti)

def get_bcc0300(bcc0300):
    
    if pd.isnull(bcc0300):
        return 99
    
    bcc0300 = int(bcc0300)
    if (bcc0300 >= 25):
        return 30
    
    return bcc0300

def get_ndi_ratio(ndi_ratio):
    
    if pd.isnull(ndi_ratio):
        return 5
    
    ndi_ratio = float(ndi_ratio)
    ndi_ratio = np.floor(ndi_ratio)
    
    if (ndi_ratio < 10):
        return 5
    if (ndi_ratio > 75):
        return 80
    
    return ndi_ratio  



In [ ]:

import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

import library_binning as lib
#import scoringData as sd

def score_collector_phase1(lead, debug = False):
    
    mod_lead = pd.DataFrame()
    bin_vars = {}
    error = False
    error_msg = ''
    prob_prediction = 0
    npv = 0
    MODEL_NAME = 'fplus_ls_201902_v1'
    
    try:
        # create binned/transformed variables
        bin_vars['co_app'] = pd.Series(lib.get_co_app_cat(lead['co_app_verifiable_annual_income']))
        bin_vars['loan_use'] = lib.get_loan_use_cat(lead['loan_use'])
        bin_vars['employment'] = lib.get_employment_cat(lead['employment_status'])
        bin_vars['loan_amount'] = lib.get_loan_amount_cat(lead['final_loan_amount'])
        bin_vars['mkt_chan'] = lib.get_mkt_chan_cat(lead['utm_source'])
        bin_vars['ficox'] = lib.get_fico(lead['fico'])
        bin_vars['lti'] = lib.get_lti(lead['lti'])
        bin_vars['bcc0300'] = lib.get_bcc0300(lead['xpn_bcc0300'])
        bin_vars['ndi'] = lib.get_ndi_ratio(lead['ndi_ratio'])
        bin_vars['ndisq'] = bin_vars['ndi'] * bin_vars['ndi']
        
        mod_lead = pd.DataFrame(bin_vars)
        
        # create dummies
        cat_vars=['co_app','loan_use','employment','loan_amount','mkt_chan']
        for var in cat_vars:
            cat_list = pd.get_dummies(mod_lead[var], prefix=var)
            temp=mod_lead.join(cat_list)
            mod_lead=temp
        data_vars=mod_lead.columns.values.tolist()
        to_keep=[i for i in data_vars if i not in cat_vars]
        mod_lead=mod_lead[to_keep]
        # print(mod_lead.columns.values)
        
        # re-index to have same columns as the model
        mod_lead = mod_lead.reindex(columns = ['ficox', 'lti', 'bcc0300', 'ndi', 'ndisq', 'co_app_0', 'co_app_1',
                                       'loan_use_1', 'loan_use_2', 'loan_use_3', 'loan_use_4', 'employment_1',
                                       'employment_2', 'employment_3', 'employment_4', 'loan_amount_1',
                                       'loan_amount_2', 'loan_amount_3', 'loan_amount_4', 'mkt_chan_1',
                                       'mkt_chan_2', 'mkt_chan_3', 'mkt_chan_5', 'mkt_chan_6', 
                                       'mkt_chan_7'], fill_value=0)
        
        # score
        #prob_prediction = sd.fplus_ls_model.predict_proba(mod_lead)[0][1] I'm changing this to remove sd and just loading scoring model and data already
        prob_prediction = fplus_ls_model.predict_proba(mod_lead)[0][1]
        #print(prob_prediction)
        
        # calc NPV
        npv = get_npv_calc(lead['final_loan_amount'], prob_prediction, lead['utm_source'])
        
    except Exception as e:
        # print(e)
        error = True
        error_msg = 'Error in Scoring.'
    
    # add an error flag
    if (debug):
        return_scores = {
            'bin_lead': mod_lead,
            'fuse.score':prob_prediction,
            'fuse.npv': npv,
            'fuse.model': MODEL_NAME,
            'fuse.error_flag': error,
            'fuse.error_reason': error_msg
        }
        return return_scores
    
    # return the dictionary
    return_scores = {
        'fuse.score':prob_prediction,
        'fuse.npv': npv,
        'fuse.model': MODEL_NAME,
        'fuse.error_flag': error,
        'fuse.error_reason': error_msg
    }
    return return_scores


def get_npv_calc(loan_amt,prob_prediction,utm_source):
    
    if pd.notnull(utm_source):
        utm_src = utm_source.strip()
        utm_src = utm_src.lower()
    
    npv = 0
    cpa = 0
    
    #if pd.notnull(utm_source) and (utm_src in set(sd.unitEconomicsData['utm_source'])):   commenting this out because of the sd and loading it locally
    #    cpa = sd.unitEconomicsData.loc[sd.unitEconomicsData['utm_source'] == utm_src, 'CPA'].values[0]
    if pd.notnull(utm_source) and (utm_src in set(unitEconomicsData['utm_source'])):
        cpa = unitEconomicsData.loc[unitEconomicsData['utm_source'] == utm_src, 'CPA'].values[0]
                
    if pd.isnull(loan_amt):
        loan_amt = 0
    else:
        loan_amt = float(loan_amt) 
        
    npv = prob_prediction * ((loan_amt * 0.0785) - (707.9 + cpa))
    
    return npv



## Main Lead Level

In [ ]:

import numpy as np
import pandas as pd
import score_collector_phase1 as sc1

def main_leadlevel(leads, phase=1, provide="scored_lead"):
    
    if phase == 1:
        checked_lead = check_lead_phase1(leads)
        
        if checked_lead["error"]:
            return_value = {
                "fuse.score" : 0,
                "fuse.npv" : 0,
                "fuse.model" : '',
                "fuse.error_flag": True,
                "fuse_error_reason" : "Lead Validation Failed."
            }
            return return_value
        error_msg = checked_lead["warning"]
        
        return_value = sc1.score_collector_phase1(checked_lead["return_lead"])
        if len(error_msg.strip()) > 0:
            return_value['fuse.error_reason'] = error_msg + ' ' + return_value['fuse.error_reason']
        
        return return_value

# Check that the lead has all the required fields
# if a value is missing, return warning message
def check_lead_phase1(lead):
    
    # convert all keys to lower case
    raw_lead = {k.lower(): v for k, v in lead.items()}
    
    error = False
    return_lead = {}
    missing_variables = []
    error_msg = ''
    
    required_fields = ["co_app_verifiable_annual_income","loan_use","employment_status",
                       "final_loan_amount","fico","lti","xpn_bcc0300","ndi_ratio","utm_source"]

    try:
        for field in required_fields:
            if (field not in raw_lead):
                raw_lead[field] = np.NaN
        
        for key in raw_lead:
            var = returnNaNifEmptyorNaN(raw_lead[key])
            return_lead[key] = var
            if (pd.isnull(return_lead[key])):
                missing_variables.append(key) 
        
        if (len(missing_variables)>0):
            error_msg = ",".join(missing_variables)
            error_msg = "Imputing values for: " + error_msg
            
    except Exception as e: 
        #print(e)
        error=True
    
    return_value = {
        "return_lead" : return_lead,
        "warning": error_msg,
        "error": error
    }
    
    return return_value

def returnNaNifEmptyorNaN(variable):
    
    if isinstance(variable, int):
        return variable
    if isinstance(variable, float):
        return variable
    # if null or empty
    if pd.isnull(variable) or len(variable)<1:
        return np.NaN
    if variable.isnumeric():
        return variable

    return variable.strip()



In [ ]:
lead = {"co_app_verifiable_annual_income":342,"loan_use":'Debt Consolidation', \
        "employment_status":'Full-time', "final_loan_amount":25000, "fico":700,\
        "lti":25.8, "xpn_bcc0300":42, "ndi_ratio":40, "utm_source":'Lending Tree'}
check_lead_phase1(lead)

{'error': False,
 'return_lead': {'co_app_verifiable_annual_income': 342,
  'employment_status': 'Full-time',
  'fico': 700,
  'final_loan_amount': 25000,
  'loan_use': 'Debt Consolidation',
  'lti': 25.8,
  'ndi_ratio': 40,
  'utm_source': 'Lending Tree',
  'xpn_bcc0300': 42},
 'warning': ''}

## Model

In [ ]:

import scoringData as sd
import main_leadLevel as ml
import pandas as pd
import json

sd.init()

# Assumption : gets dictionary as a input
def run(leads):
    score = ml.main_leadlevel(leads)
    #score is dictionary, score['fuse_score'] is dataframe
    
    #add scores to the lead
    if ('co_app_verifiable_annual_income' in leads):
        leads['co_app_verifiable_annual_income']=str(leads['co_app_verifiable_annual_income'])
    if ('fico' in leads):
        leads['fico']=str(leads['fico'])
    if ('LTI' in leads):
        leads['LTI']=str(leads['LTI'])
    if ('xpn_bcc0300' in leads):
        leads['xpn_bcc0300']=str(leads['xpn_bcc0300'])
    if ('ndi_ratio' in leads):
        leads['ndi_ratio']=str(leads['ndi_ratio'])
    if ('final_loan_amount' in leads):
        leads['final_loan_amount']=str(leads['final_loan_amount'])
    leads.update(score)
    
    return_list = []
    return_list.append(leads)
    return return_list

def format_scores_returned(score):

    # convert everything to a dataframe
    score = pd.DataFrame.from_dict(score,orient='index').transpose() 
    
    # convert back to dictionary in list format
    score = score.to_dict('list')
    
    return score



In [ ]:
run(lead)

[{'co_app_verifiable_annual_income': '342',
  'employment_status': 'Full-time',
  'fico': '700',
  'final_loan_amount': '25000',
  'fuse.error_flag': True,
  'fuse.error_reason': 'Error in Scoring.',
  'fuse.model': 'fplus_ls_201902_v1',
  'fuse.npv': 0,
  'fuse.score': 0,
  'loan_use': 'Debt Consolidation',
  'lti': 25.8,
  'ndi_ratio': '40',
  'utm_source': 'Lending Tree',
  'xpn_bcc0300': '42'}]

In [ ]:
main_leadlevel(lead)

{'fuse.error_flag': True,
 'fuse.error_reason': 'Error in Scoring.',
 'fuse.model': 'fplus_ls_201902_v1',
 'fuse.npv': 0,
 'fuse.score': 0}

In [ ]:
score_collector_phase1(lead)

{'fuse.error_flag': False,
 'fuse.error_reason': '',
 'fuse.model': 'fplus_ls_201902_v1',
 'fuse.npv': 65.49229937961921,
 'fuse.score': 0.052201737111126426}

In [ ]:
lead

{'co_app_verifiable_annual_income': '342',
 'employment_status': 'Full-time',
 'fico': '700',
 'final_loan_amount': '25000',
 'fuse.error_flag': True,
 'fuse.error_reason': 'Error in Scoring.',
 'fuse.model': 'fplus_ls_201902_v1',
 'fuse.npv': 0,
 'fuse.score': 0,
 'loan_use': 'Debt Consolidation',
 'lti': 25.8,
 'ndi_ratio': '40',
 'utm_source': 'Lending Tree',
 'xpn_bcc0300': '42'}

# New Score

## Load Data

In [ ]:
# load data
df = pd.read_csv('full_data.csv')
df.head()

,SF_App_Id,SF_App_Date_TS,Funding_Score,Funding_Model_Segment,NPV_Score,NPV_Model_Segment,Funding,NPV_Actual,name,loan_application_id,lead_type__c,createddate,prequal_submitted_date__c,full_app_submitted_date__c,Loan_Officer_Assigned,date_funded__c,funded_flag,loan_officer_name__c,amount_of_loan_requested__c,first_amount_of_loan_requested,verifiable_annual_income__c,first_income_p1,co_app_verifiable_annual_income__c,first_income_p2,co_app,first_coapp,income_sum,first_income_sum,c_LTI,First_LTI,loan_use__c,first_loan_use,employment_status__c,ndi_ratio__c,first_ndi_ratio__c,fico__c,first_FICO,utm_source__c,bcc0300__c,first_bcc0300__c,interest_rate__c,risk_group__c,final_risk_group__c,risk_group_p1__c,NPV_Actual_Funded,NPV_Score_Funded,delta_coapp,delta_LTI,delta_NDI,delta_FICO,delta_loan_size,delta_bcc0300
0,a010f00000Vy85MAAR,2019-04-08 23:29:35,9.801852e-08,10,NaN,10,0,NaN,App-04315517,NaN,Web,2019-04-08 23:29:35,NaN,NaN,1.0,NaN,0.0,Adrian Quinonez,NaN,22000.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.441415,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
1,a010f00000WN27oAAD,2019-04-29 13:24:25,6.886856e-01,1,53.655499,3,1,77.91,App-04502943,3364269.0,Web,2019-04-29 13:24:25,2019-04-29 13:24:27,2019-04-29 18:30:36,1.0,2019-04-30,1.0,Casey Mamoe,10000.0,10000.0,11856.0,62000.0,55000.0,10000.0,1.0,1.0,66856.0,72000.0,0.149575,0.138889,Credit Card Refinancing,Debt Consolidation,Retired,62.3,0.457451,678.0,653.0,115,9.0,9.0,13.49,1.0,1.0,1.0,77.91,53.655499,0.0,-0.010686,-61.842549,-25.0,0.0,0.0
2,a010f00000Tq9b3AAB,2019-02-27 22:42:24,8.452948e-02,4,72.941334,2,0,0.00,App-03960803,3010059.0,Web,2019-02-27 22:42:24,2019-02-27 22:42:25,NaN,1.0,NaN,0.0,Jamie Boldt,20000.0,20000.0,70000.0,70000.0,NaN,NaN,0.0,0.0,70000.0,70000.0,0.285714,0.285714,Debt Consolidation,Debt Consolidation,Retired,29.0,0.290281,719.0,719.0,115,9.0,9.0,NaN,4.0,4.0,NaN,NaN,72.941334,0.0,0.000000,-28.709719,0.0,0.0,0.0
3,a010f00000WSBY6AAP,2019-05-02 23:32:29,5.279838e-01,1,41.135220,3,0,0.00,App-04548298,3390695.0,Web,2019-05-02 23:32:29,2019-05-02 23:32:31,NaN,1.0,NaN,0.0,Amanda Fuller,10000.0,10000.0,60000.0,60000.0,12000.0,12000.0,1.0,1.0,72000.0,72000.0,0.138889,0.138889,Other,Other,Full-time,44.6,0.446276,655.0,655.0,115,3.0,3.0,NaN,12.0,12.0,NaN,NaN,41.135220,0.0,0.000000,-44.153724,0.0,0.0,0.0
4,a010f00000WsXcoAAF,2019-05-30 01:40:30,4.749825e-01,1,409.867184,1,0,0.00,App-04816148,1434836.0,Web,2019-05-30 01:40:30,2019-05-30 01:40:32,NaN,1.0,NaN,0.0,Ervin Ramos,20000.0,20000.0,60000.0,60000.0,76000.0,76000.0,1.0,1.0,136000.0,136000.0,0.147059,0.147059,Debt Consolidation,Debt Consolidation,Retired,28.1,0.281249,706.0,706.0,115,12.0,11.0,NaN,3.0,3.0,NaN,NaN,409.867184,0.0,0.000000,-27.818751,0.0,0.0,-1.0


In [ ]:
df.columns

Index(['SF_App_Id', 'SF_App_Date_TS', 'Funding_Score', 'Funding_Model_Segment',
       'NPV_Score', 'NPV_Model_Segment', 'Funding', 'NPV_Actual', 'name',
       'loan_application_id', 'lead_type__c', 'createddate',
       'prequal_submitted_date__c', 'full_app_submitted_date__c',
       'Loan_Officer_Assigned', 'date_funded__c', 'funded_flag',
       'loan_officer_name__c', 'amount_of_loan_requested__c',
       'first_amount_of_loan_requested', 'verifiable_annual_income__c',
       'first_income_p1', 'co_app_verifiable_annual_income__c',
       'first_income_p2', 'co_app', 'first_coapp', 'income_sum',
       'first_income_sum', 'c_LTI', 'First_LTI', 'loan_use__c',
       'first_loan_use', 'employment_status__c', 'ndi_ratio__c',
       'first_ndi_ratio__c', 'fico__c', 'first_FICO', 'utm_source__c',
       'bcc0300__c', 'first_bcc0300__c', 'interest_rate__c', 'risk_group__c',
       'final_risk_group__c', 'risk_group_p1__c', 'NPV_Actual_Funded',
       'NPV_Score_Funded', 'delta_coapp', 

In [ ]:
lead = {"co_app_verifiable_annual_income":342,"loan_use":'Debt Consolidation', \
        "employment_status":'Full-time', "final_loan_amount":25000, "fico":700,\
        "lti":18.8, "xpn_bcc0300":42, "ndi_ratio":40, "utm_source":'Lending Tree'}

## Split Final DF and First DF

In [ ]:
final_var = ['SF_App_Id', 'Funding_Score', 'Funding_Model_Segment', 'NPV_Score',\
            'NPV_Model_Segment', 'Funding', 'NPV_Actual', 'co_app', 'loan_use__c',\
            'employment_status__c', 'amount_of_loan_requested__c', 'fico__c', \
            'c_LTI', 'bcc0300__c', 'ndi_ratio__c', 'utm_source__c']

first_var = ['SF_App_Id', 'Funding_Score', 'Funding_Model_Segment', 'NPV_Score',\
            'NPV_Model_Segment', 'Funding', 'NPV_Actual', 'first_coapp', 'first_loan_use',\
            'employment_status__c', 'first_amount_of_loan_requested', 'first_FICO', \
            'First_LTI', 'first_bcc0300__c', 'first_ndi_ratio__c', 'utm_source__c']

In [ ]:
final_df = df[final_var]
first_df = df[first_var]

In [ ]:
print(final_df.head())
print(first_df.head())

            SF_App_Id  Funding_Score  ...  ndi_ratio__c  utm_source__c
0  a010f00000Vy85MAAR   9.801852e-08  ...           NaN            NaN
1  a010f00000WN27oAAD   6.886856e-01  ...          62.3            115
2  a010f00000Tq9b3AAB   8.452948e-02  ...          29.0            115
3  a010f00000WSBY6AAP   5.279838e-01  ...          44.6            115
4  a010f00000WsXcoAAF   4.749825e-01  ...          28.1            115

[5 rows x 16 columns]
            SF_App_Id  Funding_Score  ...  first_ndi_ratio__c  utm_source__c
0  a010f00000Vy85MAAR   9.801852e-08  ...            0.441415            NaN
1  a010f00000WN27oAAD   6.886856e-01  ...            0.457451            115
2  a010f00000Tq9b3AAB   8.452948e-02  ...            0.290281            115
3  a010f00000WSBY6AAP   5.279838e-01  ...            0.446276            115
4  a010f00000WsXcoAAF   4.749825e-01  ...            0.281249            115

[5 rows x 16 columns]


## Transform Variables

**These Metrics need to be multiplied by 100**

c_LTI

First_LTI

first_ndi_ratio__c



### Initial Transform

In [ ]:
final_df['c_LTI'] = final_df['c_LTI']*100
first_df['First_LTI'] = first_df['First_LTI']*100
first_df['first_ndi_ratio__c'] = first_df['first_ndi_ratio__c']*100

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [ ]:
first_df.describe()

,Funding_Score,Funding_Model_Segment,NPV_Score,NPV_Model_Segment,Funding,NPV_Actual,first_coapp,first_amount_of_loan_requested,first_FICO,First_LTI,first_bcc0300__c,first_ndi_ratio__c
count,2.507490e+05,250749.000000,250730.000000,250749.000000,250749.000000,250730.000000,249300.000000,248982.000000,246023.000000,248783.000000,247246.000000,244729.000000
mean,8.295375e-02,5.532856,60.030017,4.658890,0.090752,45.628623,0.153987,22188.402152,714.127659,23.702926,7.661899,37.842918
std,9.700508e-02,2.889049,117.059931,2.679194,0.287257,258.033385,0.360937,11604.587842,111.537659,15.029030,5.353731,14.959756
min,2.856693e-08,1.000000,-327.160570,1.000000,0.000000,-632.550000,0.000000,2500.000000,431.000000,0.001400,0.000000,0.003641
25%,2.256845e-02,3.000000,7.506289,2.000000,0.000000,0.000000,0.000000,13000.000000,676.000000,15.000000,4.000000,27.983692
50%,5.067782e-02,6.000000,25.367588,4.000000,0.000000,0.000000,0.000000,20000.000000,707.000000,21.052632,6.000000,36.769266
75%,1.038143e-01,8.000000,66.514386,7.000000,0.000000,0.000000,0.000000,30000.000000,745.000000,29.166667,10.000000,47.077911
max,9.477318e-01,10.000000,2137.949959,10.000000,1.000000,2432.910000,1.000000,288000.000000,9003.000000,1800.000000,99.000000,2300.065551


In [ ]:
final_df.columns

Index(['SF_App_Id', 'Funding_Score', 'Funding_Model_Segment', 'NPV_Score',
       'NPV_Model_Segment', 'Funding', 'NPV_Actual', 'co_app', 'loan_use__c',
       'employment_status__c', 'amount_of_loan_requested__c', 'fico__c',
       'c_LTI', 'bcc0300__c', 'ndi_ratio__c', 'utm_source__c'],
      dtype='object')

### Apply Model Transformations

In [ ]:
# Transform variables
final_df['co_app'] = final_df['co_app'].apply(get_co_app_cat)
final_df['loan_use'] = final_df['loan_use__c'].apply(get_loan_use_cat)
final_df['employment'] = final_df['employment_status__c'].apply(get_employment_cat)
final_df['loan_amount'] = final_df['amount_of_loan_requested__c'].apply(get_loan_amount_cat)
final_df['mkt_chan'] = final_df['utm_source__c'].apply(get_mkt_chan_cat)
final_df['ficox'] = final_df['fico__c'].apply(get_fico)
final_df['lti'] = final_df['c_LTI'].apply(get_lti)
final_df['bcc0300'] = final_df['bcc0300__c'].apply(get_bcc0300)
final_df['ndi'] = final_df['ndi_ratio__c'].apply(get_ndi_ratio)
final_df['ndisq'] = final_df['ndi'] * final_df['ndi']

# Transform variables
first_df['co_app'] = first_df['first_coapp'].apply(get_co_app_cat)
first_df['loan_use'] = first_df['first_loan_use'].apply(get_loan_use_cat)
first_df['employment'] = first_df['employment_status__c'].apply(get_employment_cat)
first_df['loan_amount'] = first_df['first_amount_of_loan_requested'].apply(get_loan_amount_cat)
first_df['mkt_chan'] = first_df['utm_source__c'].apply(get_mkt_chan_cat)
first_df['ficox'] = first_df['first_FICO'].apply(get_fico)
first_df['lti'] = first_df['First_LTI'].apply(get_lti)
first_df['bcc0300'] = first_df['first_bcc0300__c'].apply(get_bcc0300)
first_df['ndi'] = first_df['first_ndi_ratio__c'].apply(get_ndi_ratio)
first_df['ndisq'] = first_df['ndi'] * first_df['ndi']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

### Dummy Variables

#### Final DF

In [ ]:
final_df.columns

Index(['SF_App_Id', 'Funding_Score', 'Funding_Model_Segment', 'NPV_Score',
       'NPV_Model_Segment', 'Funding', 'NPV_Actual', 'co_app', 'loan_use__c',
       'employment_status__c', 'amount_of_loan_requested__c', 'fico__c',
       'c_LTI', 'bcc0300__c', 'ndi_ratio__c', 'utm_source__c', 'loan_use',
       'employment', 'loan_amount', 'mkt_chan', 'ficox', 'lti', 'bcc0300',
       'ndi', 'ndisq'],
      dtype='object')

In [ ]:
# create dummies
cat_vars = ['co_app','loan_use','employment','loan_amount','mkt_chan']
for var in cat_vars:
    cat_list = pd.get_dummies(final_df[var], prefix=var)
    temp = final_df.join(cat_list)
    final_df = temp
data_vars = final_df.columns.values.tolist()
to_keep = [i for i in data_vars if i not in cat_vars]
final_df = final_df[to_keep]

In [ ]:
final_df.columns

Index(['SF_App_Id', 'Funding_Score', 'Funding_Model_Segment', 'NPV_Score',
       'NPV_Model_Segment', 'Funding', 'NPV_Actual', 'loan_use__c',
       'employment_status__c', 'amount_of_loan_requested__c', 'fico__c',
       'c_LTI', 'bcc0300__c', 'ndi_ratio__c', 'utm_source__c', 'ficox', 'lti',
       'bcc0300', 'ndi', 'ndisq', 'co_app_0', 'co_app_1', 'loan_use_1',
       'loan_use_2', 'loan_use_3', 'loan_use_4', 'employment_1',
       'employment_2', 'employment_3', 'employment_4', 'loan_amount_1',
       'loan_amount_2', 'loan_amount_3', 'loan_amount_4', 'mkt_chan_1',
       'mkt_chan_2', 'mkt_chan_3', 'mkt_chan_5', 'mkt_chan_6', 'mkt_chan_7'],
      dtype='object')

In [ ]:
# re-index to have same columns as the model
final_df_indexed = final_df.reindex(columns = ['ficox', 'lti', 'bcc0300', 'ndi', 'ndisq', 'co_app_0', 'co_app_1',
                               'loan_use_1', 'loan_use_2', 'loan_use_3', 'loan_use_4', 'employment_1',
                               'employment_2', 'employment_3', 'employment_4', 'loan_amount_1',
                               'loan_amount_2', 'loan_amount_3', 'loan_amount_4', 'mkt_chan_1',
                               'mkt_chan_2', 'mkt_chan_3', 'mkt_chan_5', 'mkt_chan_6', 
                               'mkt_chan_7'], fill_value=0)

In [ ]:
prob_prediction = fplus_ls_model.predict_proba(final_df_indexed)[0][1]

In [ ]:
final_df['New_Funding_Score'] = fplus_ls_model.predict_proba(final_df_indexed)[:,1]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
final_df['New_NPV_Score'] = final_df.apply(lambda row : get_npv_calc(row['amount_of_loan_requested__c'], 
                                  row['New_Funding_Score'], row['utm_source__c']), axis = 1) 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


#### First DF

In [ ]:
first_df.columns

Index(['SF_App_Id', 'Funding_Score', 'Funding_Model_Segment', 'NPV_Score',
       'NPV_Model_Segment', 'Funding', 'NPV_Actual', 'first_coapp',
       'first_loan_use', 'employment_status__c',
       'first_amount_of_loan_requested', 'first_FICO', 'First_LTI',
       'first_bcc0300__c', 'first_ndi_ratio__c', 'utm_source__c', 'co_app',
       'loan_use', 'employment', 'loan_amount', 'mkt_chan', 'ficox', 'lti',
       'bcc0300', 'ndi', 'ndisq'],
      dtype='object')

In [ ]:
# create dummies
cat_vars = ['co_app','loan_use','employment','loan_amount','mkt_chan']
for var in cat_vars:
    cat_list = pd.get_dummies(first_df[var], prefix=var)
    temp = first_df.join(cat_list)
    first_df = temp
data_vars = first_df.columns.values.tolist()
to_keep = [i for i in data_vars if i not in cat_vars]
first_df = first_df[to_keep]

In [ ]:
first_df.columns

Index(['SF_App_Id', 'Funding_Score', 'Funding_Model_Segment', 'NPV_Score',
       'NPV_Model_Segment', 'Funding', 'NPV_Actual', 'first_coapp',
       'first_loan_use', 'employment_status__c',
       'first_amount_of_loan_requested', 'first_FICO', 'First_LTI',
       'first_bcc0300__c', 'first_ndi_ratio__c', 'utm_source__c', 'ficox',
       'lti', 'bcc0300', 'ndi', 'ndisq', 'co_app_0', 'co_app_1', 'loan_use_1',
       'loan_use_2', 'loan_use_3', 'loan_use_4', 'employment_1',
       'employment_2', 'employment_3', 'employment_4', 'loan_amount_1',
       'loan_amount_2', 'loan_amount_3', 'loan_amount_4', 'mkt_chan_1',
       'mkt_chan_2', 'mkt_chan_3', 'mkt_chan_5', 'mkt_chan_6', 'mkt_chan_7'],
      dtype='object')

In [ ]:
# re-index to have same columns as the model
first_df_indexed = first_df.reindex(columns = ['ficox', 'lti', 'bcc0300', 'ndi', 'ndisq', 'co_app_0', 'co_app_1',
                               'loan_use_1', 'loan_use_2', 'loan_use_3', 'loan_use_4', 'employment_1',
                               'employment_2', 'employment_3', 'employment_4', 'loan_amount_1',
                               'loan_amount_2', 'loan_amount_3', 'loan_amount_4', 'mkt_chan_1',
                               'mkt_chan_2', 'mkt_chan_3', 'mkt_chan_5', 'mkt_chan_6', 
                               'mkt_chan_7'], fill_value=0)

### Predict

In [ ]:
first_df['First_Funding_Score'] = fplus_ls_model.predict_proba(first_df_indexed)[:,1]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
first_df['New_NPV_Score'] = first_df.apply(lambda row : get_npv_calc(row['first_amount_of_loan_requested'], 
                                  row['First_Funding_Score'], row['utm_source__c']), axis = 1) 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
first_df.to_csv('first_df.csv', index=False)

In [ ]:
final_df.to_csv('final_df.csv', index=False)

In [ ]:
['SF_App_Id', 'Funding_Score', 'Funding_Model_Segment', 'NPV_Score',\
            'NPV_Model_Segment', 'Funding', 'NPV_Actual', 'co_app', 'loan_use__c',\
            'employment_status__c', 'amount_of_loan_requested__c', 'fico__c', \
            'c_LTI', 'bcc0300__c', 'ndi_ratio__c', 'utm_source__c']

# Austen Data

In [ ]:
# Using GBQ shout Out to Hughes
import pandas_gbq
import pydata_google_auth

SCOPES = [
    'https://www.googleapis.com/auth/cloud-platform',
    'https://www.googleapis.com/auth/drive',
]

credentials = pydata_google_auth.get_user_credentials(
    SCOPES,
    auth_local_webserver=False)

In [ ]:
sql = """
SELECT 
  id
, co_app_verifiable_annual_income__c
, loan_use__c
, employment_status__c
, amount_of_loan_requested__c
, fico__c
, lti__c
, bcc0300__c
, ndi_ratio__c
, utm_source__c

FROM `freedom-dw.salesforce_ffam.application__c` a 

WHERE createddate >= '2019-01-01'
"""

In [ ]:
df1 = pandas_gbq.read_gbq(sql, project_id='ffn-dw-bigquery-prd', credentials=credentials, dialect='standard')

df = df1

In [ ]:
final_df = df

In [ ]:
final_df.head()

,id,co_app_verifiable_annual_income__c,loan_use__c,employment_status__c,amount_of_loan_requested__c,fico__c,lti__c,bcc0300__c,ndi_ratio__c,utm_source__c
0,a010f00000W0aYWAAZ,NaN,Home Improvement,Full-time,20000.0,756.0,16.7,26.0,22.8,Quinstreet
1,a010f00000WTBuLAAX,NaN,Credit Card Refinancing,Full-time,20000.0,673.0,13.2,23.0,24.4,Lending Tree
2,a010f00000YkMLPAA3,NaN,Debt Consolidation,Full-time,35000.0,750.0,34.9,23.0,30.5,even
3,a010f00000WqYL7AAN,NaN,Debt Consolidation,Full-time,10000.0,631.0,13.9,24.0,156.2,even
4,a010f00000V7d3OAAR,NaN,Debt Consolidation,Retired,45000.0,639.0,34.9,21.0,66.6,crowdnetic


In [ ]:
# Transform variables
final_df['co_app'] = final_df['co_app_verifiable_annual_income__c'].apply(get_co_app_cat)
final_df['loan_use'] = final_df['loan_use__c'].apply(get_loan_use_cat)
final_df['employment'] = final_df['employment_status__c'].apply(get_employment_cat)
final_df['loan_amount'] = final_df['amount_of_loan_requested__c'].apply(get_loan_amount_cat)
final_df['mkt_chan'] = final_df['utm_source__c'].apply(get_mkt_chan_cat)
final_df['ficox'] = final_df['fico__c'].apply(get_fico)
final_df['lti'] = final_df['lti__c'].apply(get_lti)
final_df['bcc0300'] = final_df['bcc0300__c'].apply(get_bcc0300)
final_df['ndi'] = final_df['ndi_ratio__c'].apply(get_ndi_ratio)
final_df['ndisq'] = final_df['ndi'] * final_df['ndi']

In [ ]:
# create dummies
cat_vars = ['co_app','loan_use','employment','loan_amount','mkt_chan']
for var in cat_vars:
    cat_list = pd.get_dummies(final_df[var], prefix=var)
    temp = final_df.join(cat_list)
    final_df = temp
data_vars = final_df.columns.values.tolist()
to_keep = [i for i in data_vars if i not in cat_vars]
final_df = final_df[to_keep]

In [ ]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2602875 entries, 0 to 2602874
Data columns (total 35 columns):
id                                    object
co_app_verifiable_annual_income__c    float64
loan_use__c                           object
employment_status__c                  object
amount_of_loan_requested__c           float64
fico__c                               float64
lti__c                                float64
bcc0300__c                            float64
ndi_ratio__c                          float64
utm_source__c                         object
ficox                                 int64
lti                                   float64
bcc0300                               int64
ndi                                   float64
ndisq                                 float64
co_app_0                              uint8
co_app_1                              uint8
loan_use_1                            uint8
loan_use_2                            uint8
loan_use_3                   

In [ ]:
# re-index to have same columns as the model
final_df_indexed = final_df.reindex(columns = ['ficox', 'lti', 'bcc0300', 'ndi', 'ndisq', 'co_app_0', 'co_app_1',
                               'loan_use_1', 'loan_use_2', 'loan_use_3', 'loan_use_4', 'employment_1',
                               'employment_2', 'employment_3', 'employment_4', 'loan_amount_1',
                               'loan_amount_2', 'loan_amount_3', 'loan_amount_4', 'mkt_chan_1',
                               'mkt_chan_2', 'mkt_chan_3', 'mkt_chan_5', 'mkt_chan_6', 
                               'mkt_chan_7'], fill_value=0)

In [ ]:
final_df_indexed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2602875 entries, 0 to 2602874
Data columns (total 25 columns):
ficox            int64
lti              float64
bcc0300          int64
ndi              float64
ndisq            float64
co_app_0         uint8
co_app_1         uint8
loan_use_1       uint8
loan_use_2       uint8
loan_use_3       uint8
loan_use_4       uint8
employment_1     uint8
employment_2     uint8
employment_3     uint8
employment_4     uint8
loan_amount_1    uint8
loan_amount_2    uint8
loan_amount_3    uint8
loan_amount_4    uint8
mkt_chan_1       uint8
mkt_chan_2       uint8
mkt_chan_3       uint8
mkt_chan_5       uint8
mkt_chan_6       uint8
mkt_chan_7       uint8
dtypes: float64(3), int64(2), uint8(20)
memory usage: 148.9 MB


In [ ]:
prob_prediction = fplus_ls_model.predict_proba(final_df_indexed)[0][1]

In [ ]:
final_df['New_Funding_Score'] = fplus_ls_model.predict_proba(final_df_indexed)[:,1]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
final_df['New_NPV_Score'] = final_df.apply(lambda row : get_npv_calc(row['amount_of_loan_requested__c'], 
                                  row['New_Funding_Score'], row['utm_source__c']), axis = 1) 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df = final_df[['id', 'amount_of_loan_requested__c', 'utm_source__c', 'New_Funding_Score', 'New_NPV_Score']]

In [ ]:
df.head()

,id,amount_of_loan_requested__c,utm_source__c,New_Funding_Score,New_NPV_Score
0,a010f00000W0aYWAAZ,20000.0,Quinstreet,0.006548,2.833993
1,a010f00000WTBuLAAX,20000.0,Lending Tree,0.042487,36.627884
2,a010f00000YkMLPAA3,35000.0,even,0.004170,6.775986
3,a010f00000WqYL7AAN,10000.0,even,0.013293,-4.488235
4,a010f00000V7d3OAAR,45000.0,crowdnetic,0.028469,69.947452


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2602875 entries, 0 to 2602874
Data columns (total 5 columns):
id                             object
amount_of_loan_requested__c    float64
utm_source__c                  object
New_Funding_Score              float64
New_NPV_Score                  float64
dtypes: float64(3), object(2)
memory usage: 99.3+ MB


In [ ]:
sql_1 = """
SELECT 
application_key

FROM `ffam-data-platform.standardized_data.fplus_application` 

WHERE 

new_lead_datetime  >= '2019-01-01'
AND flag_eligible_lead = TRUE
AND lead_type = 'Web'
AND latest_prequal_decision <> 'DECLINED'
"""

In [ ]:
good_leads = pandas_gbq.read_gbq(sql_1, project_id='ffn-dw-bigquery-prd', credentials=credentials, dialect='standard')

In [ ]:
good_leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 697536 entries, 0 to 697535
Data columns (total 1 columns):
application_key    697536 non-null object
dtypes: object(1)
memory usage: 5.3+ MB


In [ ]:
good_leads1 = good_leads.merge(df, left_on='application_key', right_on='id')

In [ ]:
good_leads1 = good_leads1.drop(columns=['application_key'])
good_leads1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 697536 entries, 0 to 697535
Data columns (total 5 columns):
id                             697536 non-null object
amount_of_loan_requested__c    697536 non-null float64
utm_source__c                  697536 non-null object
New_Funding_Score              697536 non-null float64
New_NPV_Score                  697536 non-null float64
dtypes: float64(3), object(2)
memory usage: 31.9+ MB


In [ ]:
good_leads2 = good_leads1.drop_duplicates(subset=['id'], keep='first')
good_leads2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 697536 entries, 0 to 697535
Data columns (total 5 columns):
id                             697536 non-null object
amount_of_loan_requested__c    697536 non-null float64
utm_source__c                  697536 non-null object
New_Funding_Score              697536 non-null float64
New_NPV_Score                  697536 non-null float64
dtypes: float64(3), object(2)
memory usage: 31.9+ MB


# Send Back to GBQ

In [ ]:
#set destination table to insert data into
destinationtable = 'Jason.Austen_NPV_Fplus'
project_id='ffn-dw-bigquery-prd'

#send data to GBQ (pandas_gqb.to_gbq)
#https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
#params are: dataframe, destination_table, project_id, if_exists (append), and table_schema (list of dicts, name and type)
pandas_gbq.to_gbq(dataframe = good_leads1,
                  destination_table = destinationtable,
                  project_id = project_id,
                  if_exists = 'append',
                  table_schema = [{'name':'id',
                                   'type':'STRING'},
                                  {'name':'amount_of_loan_requested__c',
                                   'type':'FLOAT'},
                                  {'name':'utm_source__c',
                                   'type':'STRING'},
                                  {'name':'New_Funding_Score',
                                   'type':'FLOAT'},
                                  {'name':'New_NPV_Score',
                                   'type':'FLOAT'}])




0it [00:00, ?it/s]


1it [00:06,  6.88s/it]




# Shit for Aleks

In [ ]:
# load data
import pandas as pd
df = pd.read_csv('Retrain_Fplus.csv')
df.head()

,SF_App_Id,Funding,first_coapp,first_loan_use,employment_status__c,first_amount_of_loan_requested,first_FICO,First_LTI,first_bcc0300__c,first_ndi_ratio__c,utm_source__c,ficox,lti,bcc0300,ndi,ndisq,co_app_0,co_app_1,loan_use_1,loan_use_2,loan_use_3,loan_use_4,employment_1,employment_2,employment_3,employment_4,loan_amount_1,loan_amount_2,loan_amount_3,loan_amount_4,mkt_chan_1,mkt_chan_2,mkt_chan_3,mkt_chan_5,mkt_chan_6,mkt_chan_7
0,a010f00000Vy85MAAR,0,0.0,NaN,NaN,22000.0,NaN,NaN,NaN,44.141485,NaN,990,36,99,44,1936,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0
1,a010f00000WN27oAAD,1,1.0,Debt Consolidation,Retired,10000.0,653.0,13.888889,9.0,45.745143,115,653,13,9,45,2025,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0
2,a010f00000Tq9b3AAB,0,0.0,Debt Consolidation,Retired,20000.0,719.0,28.571429,9.0,29.028113,115,719,28,9,29,841,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0
3,a010f00000WSBY6AAP,0,1.0,Other,Full-time,10000.0,655.0,13.888889,3.0,44.627555,115,655,13,3,44,1936,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0
4,a010f00000WsXcoAAF,0,1.0,Debt Consolidation,Retired,20000.0,706.0,14.705882,11.0,28.124863,115,706,14,11,28,784,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0


In [ ]:
df1 = df.dropna()

In [ ]:
df1.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 244363 entries, 1 to 250748
Data columns (total 36 columns):
SF_App_Id                         244363 non-null object
Funding                           244363 non-null int64
first_coapp                       244363 non-null float64
first_loan_use                    244363 non-null object
employment_status__c              244363 non-null object
first_amount_of_loan_requested    244363 non-null float64
first_FICO                        244363 non-null float64
First_LTI                         244363 non-null float64
first_bcc0300__c                  244363 non-null float64
first_ndi_ratio__c                244363 non-null float64
utm_source__c                     244363 non-null object
ficox                             244363 non-null int64
lti                               244363 non-null int64
bcc0300                           244363 non-null int64
ndi                               244363 non-null int64
ndisq                            

In [ ]:
#set destination table to insert data into
destinationtable = 'Jason.retrain_fplus'
project_id='ffn-dw-bigquery-prd'

#send data to GBQ (pandas_gqb.to_gbq)
#https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
#params are: dataframe, destination_table, project_id, if_exists (append), and table_schema (list of dicts, name and type)
pandas_gbq.to_gbq(dataframe = df1,
                  destination_table = destinationtable,
                  project_id = project_id,
                  if_exists = 'append',
                  table_schema = [
{'name':'SF_App_Id', 'type':'STRING'},
{'name':'Funding', 'type':'INTEGER'},
{'name':'first_coapp', 'type':'INTEGER'},
{'name':'first_loan_use', 'type':'STRING'},
{'name':'employment_status__c', 'type':'STRING'},
{'name':'first_amount_of_loan_requested', 'type':'INTEGER'},
{'name':'first_FICO', 'type':'INTEGER'},
{'name':'First_LTI', 'type':'FLOAT'},
{'name':'first_bcc0300__c', 'type':'INTEGER'},
{'name':'first_ndi_ratio__c', 'type':'FLOAT'},
{'name':'utm_source__c', 'type':'INTEGER'},
{'name':'ficox', 'type':'INTEGER'},
{'name':'lti', 'type':'INTEGER'},
{'name':'bcc0300', 'type':'INTEGER'},
{'name':'ndi', 'type':'INTEGER'},
{'name':'ndisq', 'type':'INTEGER'},
{'name':'co_app_0', 'type':'INTEGER'},
{'name':'co_app_1', 'type':'INTEGER'},
{'name':'loan_use_1', 'type':'INTEGER'},
{'name':'loan_use_2', 'type':'INTEGER'},
{'name':'loan_use_3', 'type':'INTEGER'},
{'name':'loan_use_4', 'type':'INTEGER'},
{'name':'employment_1', 'type':'INTEGER'},
{'name':'employment_2', 'type':'INTEGER'},
{'name':'employment_3', 'type':'INTEGER'},
{'name':'employment_4', 'type':'INTEGER'},
{'name':'loan_amount_1', 'type':'INTEGER'},
{'name':'loan_amount_2', 'type':'INTEGER'},
{'name':'loan_amount_3', 'type':'INTEGER'},
{'name':'loan_amount_4', 'type':'INTEGER'},
{'name':'mkt_chan_1', 'type':'INTEGER'},
{'name':'mkt_chan_2', 'type':'INTEGER'},
{'name':'mkt_chan_3', 'type':'INTEGER'},
{'name':'mkt_chan_5', 'type':'INTEGER'},
{'name':'mkt_chan_6', 'type':'INTEGER'},
{'name':'mkt_chan_7', 'type':'INTEGER'}])

GenericGBQException: ignored

In [ ]:
#set destination table to insert data into
destinationtable = 'Jason.retrain'
project_id='ffn-dw-bigquery-prd'

#send data to GBQ (pandas_gqb.to_gbq)
#https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
#params are: dataframe, destination_table, project_id, if_exists (append), and table_schema (list of dicts, name and type)
pandas_gbq.to_gbq(dataframe = df1,
                  destination_table = destinationtable,
                  project_id = project_id,
                  if_exists = 'append',
                  table_schema = [
{'name':'SF_App_Id', 'type':'STRING'},
{'name':'Funding', 'type':'INTEGER'},
{'name':'first_coapp', 'type':'INTEGER'},
{'name':'first_loan_use', 'type':'STRING'},
{'name':'employment_status__c', 'type':'STRING'},
{'name':'first_amount_of_loan_requested', 'type':'INTEGER'},
{'name':'first_FICO', 'type':'INTEGER'},
{'name':'First_LTI', 'type':'FLOAT'},
{'name':'first_bcc0300__c', 'type':'INTEGER'},
{'name':'first_ndi_ratio__c', 'type':'FLOAT'},
{'name':'utm_source__c', 'type':'INTEGER'},
{'name':'ficox', 'type':'INTEGER'},
{'name':'lti', 'type':'INTEGER'},
{'name':'bcc0300', 'type':'INTEGER'},
{'name':'ndi', 'type':'INTEGER'},
{'name':'ndisq', 'type':'INTEGER'},
{'name':'co_app_0', 'type':'INTEGER'},
{'name':'co_app_1', 'type':'INTEGER'},
{'name':'loan_use_1', 'type':'INTEGER'},
{'name':'loan_use_2', 'type':'INTEGER'},
{'name':'loan_use_3', 'type':'INTEGER'},
{'name':'loan_use_4', 'type':'INTEGER'},
{'name':'employment_1', 'type':'INTEGER'},
{'name':'employment_2', 'type':'INTEGER'},
{'name':'employment_3', 'type':'INTEGER'},
{'name':'employment_4', 'type':'INTEGER'},
{'name':'loan_amount_1', 'type':'INTEGER'},
{'name':'loan_amount_2', 'type':'INTEGER'},
{'name':'loan_amount_3', 'type':'INTEGER'},
{'name':'loan_amount_4', 'type':'INTEGER'},
{'name':'mkt_chan_1', 'type':'INTEGER'},
{'name':'mkt_chan_2', 'type':'INTEGER'},
{'name':'mkt_chan_3', 'type':'INTEGER'},
{'name':'mkt_chan_5', 'type':'INTEGER'},
{'name':'mkt_chan_6', 'type':'INTEGER'},
{'name':'mkt_chan_7', 'type':'INTEGER'}])

GenericGBQException: ignored